In [27]:
import wrds
import pandas as pd
import numpy as np
from scipy import stats
import scipy.linalg as la
from scipy.integrate import quad
from sklearn.metrics import make_scorer
from sklearn import linear_model
from sklearn import model_selection
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import linearmodels as lm
exec(open("/home/amz5/functionsByAnthony.py").read()) # do whatever is in functionsByAnthony.py
from statsmodels.iolib.summary2 import summary_col

In [28]:
conn = wrds.Connection()  

Enter your WRDS username [amz5]:amz5
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [29]:
#Downloading monthly returns from WRDS
mret = conn.raw_sql(" select * from crsp.msf""")
## Formatting date as such, merging mret to dat, adding year and month columns
mret["date"]=pd.DatetimeIndex(mret["date"])

In [30]:
#importing characteristics, rearranging date
dat=pd.read_csv('datashare.csv')
dat['date'] = pd.to_datetime(dat['DATE'].astype(str), format='%Y%m%d')
#importing GW variables and creating list of characteristic names
gw=pd.read_csv('GoyalWelch.csv')
charlist=pd.read_csv('characteristiclist1.csv')
clist=charlist["Acronym"].to_list()

In [31]:
dat=dat.merge(mret, how='left', left_on=["permno", "date"], right_on=["permno", "date"])
dat["year"]=pd.DatetimeIndex(dat["date"]).year
dat["month"]=pd.DatetimeIndex(dat["date"]).month

In [32]:
## Renaming and calculating the 8 GW variables to match what GKX use.  Merging into dat
gw['dp']=np.log(gw["D12"])-np.log(gw["Index"])#1
gw['ep']=np.log(gw["E12"])-np.log(gw["Index"])#2
gw['bm']=gw["b/m"]#3
#ntis  #4
#tbl   #5
gw['tms']=gw["lty"]-gw["tbl"]#6
gw['dfy']=gw["BAA"]-gw["AAA"]#7
#svar  #8
dat=dat.merge(gw, how='left', left_on=["year", "month"], right_on=["year", "month"]).rename(columns={"ep_y":"ep","bm_y":"bm"})

In [33]:
#drop any records with missing ret
dat=dat.dropna(subset=['ret'])
#drop any records with missing ret
dat=dat.dropna(subset=['ret'])
featlist=clist.copy()

## Replace missing values with cross-sectional monthly median
for f in featlist:
    med = dat.groupby(by=["year", "month"])[f].transform('median')
    dat[f]=np.where(dat[f].isna(), med, dat[f])
  #  print(f)

In [34]:
##Adding interactions with GW variables, and saving the names to featlist
tlist=['dp', 'ep', 'bm', 'ntis', 'tbl', "tms", "dfy", "svar"]
for c in clist:
    for t in tlist:
        dat[c+"*"+t]=dat[c]*dat[t]
        featlist=featlist+[c+"*"+t]
dat=pd.concat([dat,pd.get_dummies(dat['sic2'])], axis=1)
featlist=featlist+pd.get_dummies(dat['sic2']).columns.tolist()

<ipython-input-34-190ccbcbddb2>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dat[c+"*"+t]=dat[c]*dat[t]


In [35]:
import sklearn.linear_model as lm# import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale 
from sklearn.cross_decomposition import PLSRegression
from itertools import product
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from datetime import datetime

three=["mvel1", "bm", 'mom12m']

## End of data prep.  Now begins the ML
mintsize=18
valsize=12
lastyear=2020
firstyearOOS=1987
dat=dat[dat["year"]<=lastyear]
step=5
#range(firstyearOOS, lastyear)
## in the below, will swap out firstyearOOS for t in loop above

In [26]:
perfall=pd.DataFrame()
comp=pd.DataFrame(index=range(firstyearOOS, lastyear, step), columns=["PLSComp", "RFDepth"])

for t in range(firstyearOOS, lastyear, step):
    print(t)
    train_x=dat[(dat["year"]<t-valsize)][featlist]
    train_x=train_x.fillna(train_x.median())
    train_y=dat[(dat["year"]<t-valsize)]["ret"]
    train_x=train_x.dropna(axis='columns')

    featlist1=train_x.columns

    valid_x=dat[(dat["year"]>=t-valsize)&(dat["year"]<t)][featlist1]
    oostest_x=dat[(dat["year"]>=t)&(dat["year"]<t+step)][featlist1]
    valid_y=dat[(dat["year"]>=t-valsize)&(dat["year"]<t)]["ret"]
    oostest_y=dat[(dat["year"]>=t)&(dat["year"]<t+step)]["ret"]
    oostest_names=dat[(dat["year"]>=t)&(dat["year"]<t+step)][["permno", "month", "year", "mvel1"]]
    print("Current Time =", datetime.now().strftime("%H:%M:%S"))
  
    ### OLS-3 +H
    parameters = {'epsilon':[1, 1.15, 1.3, 1.5]}
    mselist=[]
    for i in range(0,len(parameters['epsilon'])):
        model = lm.HuberRegressor(epsilon=parameters['epsilon'][i])
        model.fit(sm.add_constant(train_x[three]), train_y)
        mse=mean_squared_error(valid_y, model.predict(sm.add_constant(valid_x[three])))
        mselist=mselist+[mse]
        print('OLS-3+H'+":"+str(i)+':done')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"))
    besti=mselist.index(max(mselist))
    model = lm.HuberRegressor(epsilon=parameters['epsilon'][besti])
    model.fit(sm.add_constant(train_x[three]), train_y)
    t1=model.predict(sm.add_constant(oostest_x[three]))
    perf=pd.concat([oostest_names.reset_index(),pd.DataFrame([np.array(oostest_y),t1]).T],
                   axis=1).rename(columns={0:'real', 1:'OLS-3+H'})
    #perf["year"]=t

    ### PLS
    parameters = {'k':[1, 2, 3, 4, 5, 6, 7]}
    mselist=[]
    for i in range(0,len(parameters['k'])):
        model = PLSRegression(n_components=parameters['k'][i])
        model.fit(scale(train_x), train_y)
        mse=mean_squared_error(valid_y, model.predict(scale(valid_x)))
        mselist=mselist+[mse]
        print('PLS'+":"+str(i)+':done')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"))
    besti=mselist.index(max(mselist))
    model = PLSRegression(n_components=parameters['k'][besti])
    model.fit(scale(train_x), train_y)
    t1=model.predict(scale(oostest_x))
    perf=pd.concat([perf,pd.DataFrame(t1)], axis=1).rename(columns={0:'PLS'})
    #perfall=perfall.append(perf)
    comp.at[t, 'PLSComp'] =parameters['k'][besti] 

    ### Elastic Net - should be +H
    x_train_scaled=sklearn.preprocessing.scale(train_x, axis=0, with_mean=True, with_std=True, copy=True)

    x_test_scaled=sklearn.preprocessing.scale(oostest_x, axis=0, with_mean=True, with_std=True, copy=True)

    x_valid_scaled=sklearn.preprocessing.scale(valid_x, axis=0, with_mean=True, with_std=True, copy=True)

    parameters = {'rho':[.5], 'lambd':[10**-4,10**-3,10**-2,10**-1]}
    mselist=[]

    for j in list(product(*parameters.values())):
        elasticModel=sklearn.linear_model.ElasticNet(alpha=j[1],l1_ratio=j[0],fit_intercept=True)
        elasticModel.fit(x_train_scaled,train_y)
        mse=mean_squared_error(valid_y, elasticModel.predict(x_valid_scaled))
        mselist=mselist+[mse]
    bestj=mselist.index(max(mselist))
    elasticModel=sklearn.linear_model.ElasticNet(alpha=list(product(*parameters.values()))[bestj][1],
                                                 l1_ratio=list(product(*parameters.values()))[bestj][0],
                                                 fit_intercept=True)
    elasticModel.fit(x_train_scaled,train_y)
    t1=elasticModel.predict(x_test_scaled)
    perf=pd.concat([perf,pd.DataFrame(t1)], axis=1).rename(columns={0:'E-Net'})
    comp.at[t, 'ENetChar'] =(~(elasticModel.coef_==0)).sum()

   
    #RF 
    parameters = {'n_estimators': [300],
                   'max_features': [10,20,30, 50],
                   'max_depth': [1,2,3,4,5,6]}
    mselist=[]
    for j in list(product(*parameters.values())):
        model = RandomForestRegressor(n_estimators=j[0],max_features=j[1], max_depth=j[2])
        model.fit(train_x, train_y)
        mse=mean_squared_error(valid_y, model.predict(valid_x))
        mselist=mselist+[mse]
        print('RF'+":"+str(j[0])+':done')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"))
    bestj=mselist.index(max(mselist))
    model = RandomForestRegressor(n_estimators=list(product(*parameters.values()))[bestj][0],
                                  max_features=list(product(*parameters.values()))[bestj][1],
                                  max_depth=list(product(*parameters.values()))[bestj][2])
    model.fit(train_x, train_y)
    t1=model.predict(oostest_x)
    perf=pd.concat([perf,pd.DataFrame(t1)], axis=1).rename(columns={0:'RF'})
    comp.at[t, 'RFDepth'] =list(product(*parameters.values()))[bestj][2]
       
    perfall=perfall.append(perf)
    perfall.to_pickle('perfall.pkl')
    comp.to_pickle('comp.pkl')

1987
Current Time = 16:24:56


/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


OLS-3+H:0:done
Current Time = 16:24:59


/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


OLS-3+H:1:done
Current Time = 16:24:59


...

OLS-3+H:2:done
Current Time = 15:03:30


/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


OLS-3+H:3:done
Current Time = 15:03:47


/opt/software/Anaconda3/2021.05/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.